In [1]:
#Install any missing requirements 
!pip install -r requirements.txt
!pip install nbformat --upgrade

#Import all the neccessary libraries
from IPython.display import clear_output
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import nbformat

init_notebook_mode()         # initiate notebook for offline plot

#clear_output so that the pip output is not displayed
clear_output()

In [2]:
# AlphaVantage API key
API_KEY = "QY5BJUJF9897PJBQ"

In [3]:
# Instantiate the ScriptData class
class ScriptData:
    def __init__(self):
        """
        Initialize an empty ScriptData object.
        """
        self.api_key = API_KEY
        self.data = {}
        self.df = {}

    def fetch_intraday_data(self, script):
        """
        Fetch intraday data for the given stock ticker symbol using the AlphaVantage API.

        Parameters:
        script (str): Stock ticker symbol.

        Returns:
        None
        """
        api_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={self.api_key}"
        response = requests.get(api_url)
        data = response.json()

        # Extract intraday data
        intraday_data = data["Time Series (5min)"]

        # Store intraday data in object
        self.data[script] = intraday_data

    def convert_intraday_data(self, script):
        """
        Convert the raw intraday data for the given stock ticker symbol into a Pandas DataFrame.

        Parameters:
        script (str): Stock ticker symbol.

        Returns:
        None
        """
        formatted_data = {}
        i = 0
        # Format intraday data into dictionary
        for timestamp, data_dict in self.data[script].items():
            formatted_data[str(i)] = {
                "timestamp": timestamp,
                "open": data_dict["1. open"],
                "high": data_dict["2. high"],
                "low": data_dict["3. low"],
                "close": data_dict["4. close"],
                "volume": data_dict["5. volume"]
            }
            i += 1

        # Convert dictionary to Pandas DataFrame
        dataframe = pd.DataFrame.from_dict(formatted_data, orient="index", columns=[
            "timestamp", "open", "high", "low", "close", "volume"])

        # Store DataFrame in object
        self.df[script] = dataframe

    def __getitem__(self, key):
        """
        Get the Pandas DataFrame for the given stock ticker symbol.

        Parameters:
        key (str): Stock ticker symbol.

        Returns:
        Pandas DataFrame: DataFrame containing intraday data for the given stock ticker symbol.
        """
        return self.df[key]

    def __setitem__(self, key, value):
        """
        Set the Pandas DataFrame for the given stock ticker symbol.

        Parameters:
        key (str): Stock ticker symbol.
        value (Pandas DataFrame): DataFrame containing intraday data for the given stock ticker symbol.

        Returns:
        None
        """
        self.df[key] = value
        
    def __contains__(self, item):
        """
        Check if the given stock ticker symbol is stored in the ScriptData object.

        Parameters:
        item (str): Stock ticker symbol.

        Returns:
        bool: True if the stock ticker symbol is stored in the ScriptData object, False otherwise.
        """
        return item in self.df

In [4]:
#Indicator1 function declaration
def indicator1(df, timeperiod):
    """
    Compute a moving average for the given time period.

    Parameters:
    df (Pandas DataFrame): DataFrame containing 'timestamp' and 'close' columns.
    timeperiod (int): Number of periods to use in the moving average calculation.

    Returns:
    Pandas DataFrame: DataFrame containing 'timestamp' and 'indicator' columns.
    """
    # Compute moving average
    moving_avg = df['close'].rolling(timeperiod).mean()

    # Create result DataFrame
    result = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': moving_avg})
    return result

In [5]:
# Strategy Class description
class Strategy:
    def __init__(self, script):
        """
        Initialize a Strategy object for the given stock ticker symbol.

        Parameters:
        script (str): Stock ticker symbol.
        """
        self.script = script
        self.script_data = ScriptData()
        self.signals = None

    def get_script_data(self):
        """
        Fetch and convert intraday data for the given stock ticker symbol, and store it in the Strategy object.

        Returns:
        None
        """
        self.script_data.fetch_intraday_data(self.script)
        self.script_data.convert_intraday_data(self.script)
        self.df = self.script_data[self.script]

    def compute_indicator_data(self, timeperiod):
        """
        Compute indicator data for the given time period.

        Parameters:
        timeperiod (int): Number of periods to use in the indicator calculation.

        Returns:
        None
        """
        self.indicator_data = indicator1(self.df, timeperiod)

    def get_signals(self):
        """
        Generate buy and sell signals based on the indicator data and current price.
        """
        # If indicators have not been computer then compute them
        self.compute_indicator_data(5)

        self.signals = pd.DataFrame(columns=['timestamp', 'signal'])

        # Iterate through each row in the DataFrame
        for i in range(1, len(self.df)):
            # Get current and previous indicator and price values and cast them to float and store them
            indicator = float(self.indicator_data['indicator'][i])
            close = float(self.df['close'][i])
            indicator_prev = float(self.indicator_data['indicator'][i-1])
            close_prev = float(self.df['close'][i-1])

            # Generate buy and sell signals
            if indicator > close and indicator_prev <= close_prev:  # Indicator cuts close upwards
                # Create empty signal entry for index i with the timestamp of the current position
                self.signals.loc[i, 'timestamp'] = self.df['timestamp'][i]
                # Signal "BUY" if indicator cuts close upwards
                self.signals.loc[i, 'signal'] = 'BUY'
            elif indicator < close and indicator_prev >= close_prev:  # Indicator cuts close downwards
                # Create empty signal entry for index i with the timestamp of the current position
                self.signals.loc[i, 'timestamp'] = self.df['timestamp'][i]
                # Signal "BUY" if indicator cuts close downwards
                self.signals.loc[i, 'signal'] = 'SELL'
            else:  # No interseection
                # Create empty signal entry for index i with the timestamp of the current position
                self.signals.loc[i, 'timestamp'] = self.df['timestamp'][i]
                # Signal "NO_SIGNAL" if there's no intersection
                self.signals.loc[i, 'signal'] = 'NO_SIGNAL'

        # Filter out rows with no signal
        return (self.signals[self.signals['signal'] != 'NO_SIGNAL'])

    def plot_strategy(self):
        """
        plots and displays a plot for history and indicators
        """

        # Make candelstick for closing history
        history_candlestick = go.Candlestick(
            x=self.df['timestamp'],
            open=self.df["open"],
            high=self.df["high"],
            low=self.df["low"],
            close=self.df["close"],
            name="Historical Data"
        )

        # Make line graph for indicator data
        indicator_line = go.Scatter(
            x=self.indicator_data['timestamp'],
            y=self.indicator_data['indicator'],
            name='SMA',
            line_color='gray'
        )

        # create a figure with the two above declared representations
        fig = go.Figure(data=[history_candlestick, indicator_line])

        # Update fig's x axs to have proper timestamp
        fig.update_layout(
            xaxis=go.layout.XAxis(
                tickmode="array",
                dtick=10,
                tickvals=self.indicator_data['timestamp'],  # actual value
                ticktext=self.indicator_data['timestamp'],  # Display text
                tickangle=90,
                tickfont=dict(size=10)
            )
        )
        iplot(fig)


In [6]:
#Initialize ScriptData class under the name 'script_data'
script_data = ScriptData()

In [7]:
#fetch, convert and display script data for GOOGL
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2022-12-22 20:00:00,87.6000,87.6400,87.5000,87.5500,2675
1,2022-12-22 19:55:00,87.6000,87.6000,87.6000,87.6000,690
2,2022-12-22 19:50:00,87.6300,87.6300,87.6300,87.6300,504
3,2022-12-22 19:45:00,87.6600,87.6600,87.6600,87.6600,402
4,2022-12-22 19:40:00,87.6300,87.6300,87.6300,87.6300,158
...,...,...,...,...,...,...
95,2022-12-22 11:30:00,87.2650,87.3199,87.2090,87.2350,182750
96,2022-12-22 11:25:00,87.2650,87.3800,87.2400,87.2600,160993
97,2022-12-22 11:20:00,87.3400,87.4458,87.2631,87.2650,185157
98,2022-12-22 11:15:00,87.3800,87.4250,87.2300,87.3400,209501


In [8]:
#fetch, convert and display script data for AAPL
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2022-12-22 20:00:00,131.4000,131.4000,131.2500,131.3000,12729
1,2022-12-22 19:55:00,131.5400,131.5500,131.3500,131.3900,15913
2,2022-12-22 19:50:00,131.5400,131.5700,131.5300,131.5401,5836
3,2022-12-22 19:45:00,131.6100,131.6100,131.5100,131.5500,8016
4,2022-12-22 19:40:00,131.6500,131.7000,131.6000,131.6000,3648
...,...,...,...,...,...,...
95,2022-12-22 12:05:00,131.7010,131.7750,131.5401,131.5800,1171454
96,2022-12-22 12:00:00,131.5200,131.7600,131.4300,131.7050,568220
97,2022-12-22 11:55:00,131.2500,131.5200,131.2100,131.5101,547869
98,2022-12-22 11:50:00,131.2800,131.3101,131.0501,131.2584,740969


In [9]:
#check if data for "GOOGL" exist for the script_data instance
'GOOGL' in script_data

True

In [10]:
#check if data for "AAPL" exist for the script_data instance
'AAPL' in script_data

True

In [11]:
#check if data for "NVDA" exist for the script_data instance
'NVDA' in script_data

False

In [12]:
#Get indicators for "GOOGL" with timeperiod of 5
indicator1(script_data['GOOGL'], timeperiod=5)

,timestamp,indicator
0,2022-12-22 20:00:00,NaN
1,2022-12-22 19:55:00,NaN
2,2022-12-22 19:50:00,NaN
3,2022-12-22 19:45:00,NaN
4,2022-12-22 19:40:00,87.61400
...,...,...
95,2022-12-22 11:30:00,87.19386
96,2022-12-22 11:25:00,87.21786
97,2022-12-22 11:20:00,87.23250
98,2022-12-22 11:15:00,87.25050


In [13]:
#Get indicators for "AAPL" with timeperiod of 5
indicator1(script_data['AAPL'], timeperiod=5)

,timestamp,indicator
0,2022-12-22 20:00:00,NaN
1,2022-12-22 19:55:00,NaN
2,2022-12-22 19:50:00,NaN
3,2022-12-22 19:45:00,NaN
4,2022-12-22 19:40:00,131.47602
...,...,...
95,2022-12-22 12:05:00,131.47036
96,2022-12-22 12:00:00,131.59926
97,2022-12-22 11:55:00,131.59764
98,2022-12-22 11:50:00,131.54832


In [14]:
#use Strategy class and show signals for "NVDA"
strategy=Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
12,2022-12-22 18:55:00,BUY
13,2022-12-22 18:50:00,SELL
15,2022-12-22 18:40:00,BUY
17,2022-12-22 18:30:00,SELL
23,2022-12-22 18:00:00,BUY
24,2022-12-22 17:55:00,SELL
26,2022-12-22 17:45:00,BUY
32,2022-12-22 17:15:00,SELL
33,2022-12-22 17:10:00,BUY
35,2022-12-22 17:00:00,SELL


In [15]:
#Plot a candlestick chart of ‘df and ‘indicator’ using the plot_strategy method on the strategy instance
strategy.plot_strategy()